In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import argparse
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
import numpy as np

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 16

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

# Flatten the data
def flatten_image(image, label):
    return tf.reshape(image, (-1, 28*28)), label

# Apply flatten function to train and test datasets
ds_train = ds_train.map(flatten_image)
ds_test = ds_test.map(flatten_image)


# **original weights:**

In [ ]:
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 32, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)


Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 28s 6ms/step - accuracy: 0.8685 - loss: 0.4993 - val_accuracy: 0.9612 - val_loss: 0.1260
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.9695 - loss: 0.1063 - val_accuracy: 0.9688 - val_loss: 0.1085
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.9795 - loss: 0.0666 - val_accuracy: 0.9766 - val_loss: 0.0743
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9863 - loss: 0.0464 - val_accuracy: 0.9772 - val_loss: 0.0723
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.9897 - loss: 0.0340 - val_accuracy: 0.9733 - val_loss: 0.0843


# **Updating weights of Layer 1:**

In [ ]:
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [8 , 4 , 2]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# # Get the initial weights of the model
initial_weights = model.get_weights()

# Initialize random weights for the first layer only
first_layer_weights = initial_weights[0]  # First layer weights
first_layer_biases = initial_weights[1]   # First layer biases


# Set random values for the first layer weights using numpy
random_weights = np.random.rand(*first_layer_weights.shape)

# Update the first layer weights to the random values while keeping the biases the same
initial_weights[0] = random_weights  # Replace the first layer's weights with random values

# Assign the modified weights back to the model
model.set_weights(initial_weights)

In [ ]:
print(first_layer_weights.shape)

(784, 256)


In [ ]:
print(first_layer_biases.shape)

(256,)


In [ ]:
model.build()

# Printing the input and output dimensions of each layer
for i, layer in enumerate(model.layers):
    weights_shape = layer.get_weights()[0].shape  # Shape of weight matrix (input_dim, output_dim)
    biases_shape = layer.get_weights()[1].shape   # Shape of bias vector (output_dim,)
    print(f"Layer {i+1}:")
    print(f"  Input shape: {weights_shape[0]}, Output shape: {weights_shape[1]}")
    print(f"  Weight matrix shape: {weights_shape}")
    print(f"  Bias vector shape: {biases_shape}\n")

Layer 1:
  Input shape: 784, Output shape: 256
  Weight matrix shape: (784, 256)
  Bias vector shape: (256,)

Layer 2:
  Input shape: 256, Output shape: 32
  Weight matrix shape: (256, 32)
  Bias vector shape: (32,)

Layer 3:
  Input shape: 32, Output shape: 10
  Weight matrix shape: (32, 10)
  Bias vector shape: (10,)



In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)


Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.1072 - loss: 2.3095 - val_accuracy: 0.1135 - val_loss: 2.3064
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.1062 - loss: 2.3068 - val_accuracy: 0.1032 - val_loss: 2.3077
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.1044 - loss: 2.3068 - val_accuracy: 0.1032 - val_loss: 2.3067
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.1028 - loss: 2.3070 - val_accuracy: 0.1135 - val_loss: 2.3036
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - accuracy: 0.1024 - loss: 2.3070 - val_accuracy: 0.1135 - val_loss: 2.3148


# **Updating weights of Layer 2:**

In [ ]:
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# Get the initial weights of the model
initial_weights = model.get_weights()

# Initialize random weights for the second layer only
second_layer_weights = initial_weights[2]  # Second layer weights
second_layer_biases = initial_weights[3]   # Second layer biases

# Set random values for the second layer weights using numpy
random_weights = np.random.rand(*second_layer_weights.shape)

# Update the second layer weights to the random values while keeping the biases the same
initial_weights[2] = random_weights  # Replace the second layer's weights with random values

# Assign the modified weights back to the model
model.set_weights(initial_weights)

# Verifying the modified weights for the second layer
# print("Updated second layer weights with random values:")
# print(initial_weights[2])  # Print the modified second layer weights

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)


Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.1043 - loss: 2.3234 - val_accuracy: 0.1028 - val_loss: 2.3080
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 38s 5ms/step - accuracy: 0.1022 - loss: 2.3159 - val_accuracy: 0.0982 - val_loss: 2.3258
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.1023 - loss: 2.3181 - val_accuracy: 0.0958 - val_loss: 2.3098
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 40s 5ms/step - accuracy: 0.1027 - loss: 2.3177 - val_accuracy: 0.0974 - val_loss: 2.3294
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.1056 - loss: 2.3161 - val_accuracy: 0.1028 - val_loss: 2.3121


# **Updating weights of Layer 3:**

In [ ]:
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# # Get the initial weights of the model
initial_weights = model.get_weights()

# # Initialize random weights for the third layer only
third_layer_weights = initial_weights[4]  # Third layer weights
third_layer_biases = initial_weights[5]   # Third layer biases

# # Set random values for the third layer weights using numpy
random_weights = np.random.randn(*third_layer_weights.shape)

# # Update the third layer weights to the random values while keeping the biases the same
initial_weights[4] = random_weights  # Replace the third layer's weights with random values

# # Assign the modified weights back to the model
model.set_weights(initial_weights)

# # Verifying the modified weights for the third layer
# print("Updated third layer weights with random values:")
# print(initial_weights[4])  # Print the modified third layer weights

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)


Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.8781 - loss: 0.4080 - val_accuracy: 0.9673 - val_loss: 0.1094
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9713 - loss: 0.0949 - val_accuracy: 0.9733 - val_loss: 0.0869
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.9811 - loss: 0.0617 - val_accuracy: 0.9737 - val_loss: 0.0898
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.9859 - loss: 0.0432 - val_accuracy: 0.9755 - val_loss: 0.0852
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.9897 - loss: 0.0313 - val_accuracy: 0.9738 - val_loss: 0.0877


# **Hamming distance 2 on layer 1(exponent):**

In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# Get the initial weights of the model
initial_weights = model.get_weights()

# Function to decompose a float into sign, exponent, and mantissa
def decompose_float(value):

    # Convert float to 64-bit binary
    binary_repr = format(struct.unpack('!Q', struct.pack('!d', value))[0], '064b')
    sign = binary_repr[0]
    exponent = binary_repr[1:12]  # 11 bits for exponent in IEEE 754
    mantissa = binary_repr[12:]   # 52 bits for mantissa
    return sign, exponent, mantissa

# Function to reconstruct a float from sign, exponent, and mantissa
def reconstruct_float(sign, exponent, mantissa):
    # Reconstruct the binary string
    binary_repr = sign + exponent + mantissa
    # Convert back to float
    return struct.unpack('!d', struct.pack('!Q', int(binary_repr, 2)))[0]

# Function to flip bits in the exponent part to achieve Hamming distance
def flip_bits_exponent(exponent_str, n_flips):
    # Convert the exponent string to a list of characters
    exponent_list = list(exponent_str)

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(exponent_list), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        exponent_list[idx] = '1' if exponent_list[idx] == '0' else '0'

    # Join the list back into a string
    return ''.join(exponent_list)

# Function to modify weights by flipping bits in the exponent only
def modify_exponent_hamming_distance(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight to ensure a Hamming distance on the exponent
    for i in range(len(flat_weights)):
        # Decompose the float into sign, exponent, and mantissa
        sign, exponent, mantissa = decompose_float(flat_weights[i])

        # Flip bits in the exponent to achieve the specified Hamming distance
        modified_exponent = flip_bits_exponent(exponent, hamming_distance)

        # Reconstruct the float using the modified exponent and original sign/mantissa
        flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)

    # Reshape back to original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the first layer (layer 1)
layer_index = 0  # Index of the first layer (0-based index)
initial_weights[layer_index * 2] = modify_exponent_hamming_distance(initial_weights[layer_index * 2], hamming_distance=2)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


<ipython-input-26-9e54dc0a62b8>:50: RuntimeWarning: overflow encountered in cast
  flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)


Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 26s 6ms/step - accuracy: 0.0999 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 46s 8ms/step - accuracy: 0.0982 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 35s 6ms/step - accuracy: 0.0998 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 6ms/step - accuracy: 0.0986 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0981 - loss: nan - val_accuracy: 0.0980 - val_loss: nan


# **Hamming distance 2 on layer 1(mantissa):**

In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# # Get the initial weights of the model
initial_weights = model.get_weights()


# Function to convert float to IEEE 754 binary (sign, exponent, mantissa)
def float_to_ieee754(value):
    # Pack the float as a binary string (64-bit)
    packed = struct.pack('>d', value)
    # Convert to 64-bit unsigned integer
    unpacked = struct.unpack('>Q', packed)[0]

    # Extract sign (1 bit), exponent (11 bits), and mantissa (52 bits)
    sign = (unpacked >> 63) & 1
    exponent = (unpacked >> 52) & 0x7FF
    mantissa = unpacked & ((1 << 52) - 1)

    return sign, exponent, mantissa

# Function to convert IEEE 754 binary (sign, exponent, mantissa) back to float
def ieee754_to_float(sign, exponent, mantissa):
    # Reassemble the binary representation
    binary_rep = (sign << 63) | (exponent << 52) | mantissa
    # Convert back to float
    packed = struct.pack('>Q', binary_rep)
    return struct.unpack('>d', packed)[0]

# Function to flip bits in the mantissa to achieve exact Hamming distance
def flip_bits_in_mantissa(mantissa, n_flips):
    # Convert mantissa to a binary string
    mantissa_bin = list(f'{mantissa:052b}')

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(mantissa_bin), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        mantissa_bin[idx] = '1' if mantissa_bin[idx] == '0' else '0'

    # Join the list back into a string and convert to integer
    return int(''.join(mantissa_bin), 2)

# Function to modify weights by flipping bits in the mantissa
def modify_weights_mantissa(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight's mantissa
    for i in range(len(flat_weights)):
        # Convert the weight to IEEE 754 components
        sign, exponent, mantissa = float_to_ieee754(flat_weights[i])

        # Flip bits in the mantissa to achieve the specified Hamming distance
        modified_mantissa = flip_bits_in_mantissa(mantissa, hamming_distance)

        # Convert the modified IEEE 754 components back to a float
        flat_weights[i] = ieee754_to_float(sign, exponent, modified_mantissa)

    # Reshape back to the original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the first layer (layer 1)
layer_index = 0  # Index of the first layer (0-based index)
initial_weights[layer_index * 2] = modify_weights_mantissa(initial_weights[layer_index * 2], hamming_distance=2)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 29s 6ms/step - accuracy: 0.8700 - loss: 0.4545 - val_accuracy: 0.9615 - val_loss: 0.1266
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - accuracy: 0.9676 - loss: 0.1059 - val_accuracy: 0.9687 - val_loss: 0.1036
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9805 - loss: 0.0653 - val_accuracy: 0.9709 - val_loss: 0.0926
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.9862 - loss: 0.0459 - val_accuracy: 0.9765 - val_loss: 0.0737
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 38s 5ms/step - accuracy: 0.9887 - loss: 0.0342 - val_accuracy: 0.9726 - val_loss: 0.0923


# **Hamming distance 4 on layer 1(exponent):**

In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# Get the initial weights of the model
initial_weights = model.get_weights()

# Function to decompose a float into sign, exponent, and mantissa
def decompose_float(value):

    # Convert float to 64-bit binary
    binary_repr = format(struct.unpack('!Q', struct.pack('!d', value))[0], '064b')
    sign = binary_repr[0]
    exponent = binary_repr[1:12]  # 11 bits for exponent in IEEE 754
    mantissa = binary_repr[12:]   # 52 bits for mantissa
    return sign, exponent, mantissa

# Function to reconstruct a float from sign, exponent, and mantissa
def reconstruct_float(sign, exponent, mantissa):
    # Reconstruct the binary string
    binary_repr = sign + exponent + mantissa
    # Convert back to float
    return struct.unpack('!d', struct.pack('!Q', int(binary_repr, 2)))[0]

# Function to flip bits in the exponent part to achieve Hamming distance
def flip_bits_exponent(exponent_str, n_flips):
    # Convert the exponent string to a list of characters
    exponent_list = list(exponent_str)

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(exponent_list), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        exponent_list[idx] = '1' if exponent_list[idx] == '0' else '0'

    # Join the list back into a string
    return ''.join(exponent_list)

# Function to modify weights by flipping bits in the exponent only
def modify_exponent_hamming_distance(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight to ensure a Hamming distance on the exponent
    for i in range(len(flat_weights)):
        # Decompose the float into sign, exponent, and mantissa
        sign, exponent, mantissa = decompose_float(flat_weights[i])

        # Flip bits in the exponent to achieve the specified Hamming distance
        modified_exponent = flip_bits_exponent(exponent, hamming_distance)

        # Reconstruct the float using the modified exponent and original sign/mantissa
        flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)

    # Reshape back to original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 0  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_exponent_hamming_distance(initial_weights[layer_index * 2], hamming_distance=4)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


<ipython-input-18-ae26e29065cc>:50: RuntimeWarning: overflow encountered in cast
  flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0996 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.1015 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.1004 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 27s 7ms/step - accuracy: 0.1003 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.0999 - loss: nan - val_accuracy: 0.0980 - val_loss: nan


# **Hamming distance 4 on layer 1(mantissa):**

In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# # Get the initial weights of the model
initial_weights = model.get_weights()


# Function to convert float to IEEE 754 binary (sign, exponent, mantissa)
def float_to_ieee754(value):
    # Pack the float as a binary string (64-bit)
    packed = struct.pack('>d', value)
    # Convert to 64-bit unsigned integer
    unpacked = struct.unpack('>Q', packed)[0]

    # Extract sign (1 bit), exponent (11 bits), and mantissa (52 bits)
    sign = (unpacked >> 63) & 1
    exponent = (unpacked >> 52) & 0x7FF
    mantissa = unpacked & ((1 << 52) - 1)

    return sign, exponent, mantissa

# Function to convert IEEE 754 binary (sign, exponent, mantissa) back to float
def ieee754_to_float(sign, exponent, mantissa):
    # Reassemble the binary representation
    binary_rep = (sign << 63) | (exponent << 52) | mantissa
    # Convert back to float
    packed = struct.pack('>Q', binary_rep)
    return struct.unpack('>d', packed)[0]

# Function to flip bits in the mantissa to achieve exact Hamming distance
def flip_bits_in_mantissa(mantissa, n_flips):
    # Convert mantissa to a binary string
    mantissa_bin = list(f'{mantissa:052b}')

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(mantissa_bin), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        mantissa_bin[idx] = '1' if mantissa_bin[idx] == '0' else '0'

    # Join the list back into a string and convert to integer
    return int(''.join(mantissa_bin), 2)

# Function to modify weights by flipping bits in the mantissa
def modify_weights_mantissa(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight's mantissa
    for i in range(len(flat_weights)):
        # Convert the weight to IEEE 754 components
        sign, exponent, mantissa = float_to_ieee754(flat_weights[i])

        # Flip bits in the mantissa to achieve the specified Hamming distance
        modified_mantissa = flip_bits_in_mantissa(mantissa, hamming_distance)

        # Convert the modified IEEE 754 components back to a float
        flat_weights[i] = ieee754_to_float(sign, exponent, modified_mantissa)

    # Reshape back to the original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 0  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_weights_mantissa(initial_weights[layer_index * 2], hamming_distance=4)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - accuracy: 0.8714 - loss: 0.4520 - val_accuracy: 0.9649 - val_loss: 0.1158
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 40s 6ms/step - accuracy: 0.9680 - loss: 0.1042 - val_accuracy: 0.9712 - val_loss: 0.0937
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.9796 - loss: 0.0650 - val_accuracy: 0.9720 - val_loss: 0.0891
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 7ms/step - accuracy: 0.9851 - loss: 0.0459 - val_accuracy: 0.9762 - val_loss: 0.0788
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - accuracy: 0.9895 - loss: 0.0341 - val_accuracy: 0.9767 - val_loss: 0.0776


# **Hamming distance 8 on layer 1(exponent):**

In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# Get the initial weights of the model
initial_weights = model.get_weights()

# Function to decompose a float into sign, exponent, and mantissa
def decompose_float(value):

    # Convert float to 64-bit binary
    binary_repr = format(struct.unpack('!Q', struct.pack('!d', value))[0], '064b')
    sign = binary_repr[0]
    exponent = binary_repr[1:12]  # 11 bits for exponent in IEEE 754
    mantissa = binary_repr[12:]   # 52 bits for mantissa
    return sign, exponent, mantissa

# Function to reconstruct a float from sign, exponent, and mantissa
def reconstruct_float(sign, exponent, mantissa):
    # Reconstruct the binary string
    binary_repr = sign + exponent + mantissa
    # Convert back to float
    return struct.unpack('!d', struct.pack('!Q', int(binary_repr, 2)))[0]

# Function to flip bits in the exponent part to achieve Hamming distance
def flip_bits_exponent(exponent_str, n_flips):
    # Convert the exponent string to a list of characters
    exponent_list = list(exponent_str)

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(exponent_list), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        exponent_list[idx] = '1' if exponent_list[idx] == '0' else '0'

    # Join the list back into a string
    return ''.join(exponent_list)

# Function to modify weights by flipping bits in the exponent only
def modify_exponent_hamming_distance(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight to ensure a Hamming distance on the exponent
    for i in range(len(flat_weights)):
        # Decompose the float into sign, exponent, and mantissa
        sign, exponent, mantissa = decompose_float(flat_weights[i])

        # Flip bits in the exponent to achieve the specified Hamming distance
        modified_exponent = flip_bits_exponent(exponent, hamming_distance)

        # Reconstruct the float using the modified exponent and original sign/mantissa
        flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)

    # Reshape back to original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 0  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_exponent_hamming_distance(initial_weights[layer_index * 2], hamming_distance=8)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


<ipython-input-27-cf45ab36ec14>:50: RuntimeWarning: overflow encountered in cast
  flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.0982 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.0995 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.0980 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.0974 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0994 - loss: nan - val_accuracy: 0.0980 - val_loss: nan


# **Hamming distance 8 on layer 1(mantissa):**

In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# # Get the initial weights of the model
initial_weights = model.get_weights()


# Function to convert float to IEEE 754 binary (sign, exponent, mantissa)
def float_to_ieee754(value):
    # Pack the float as a binary string (64-bit)
    packed = struct.pack('>d', value)
    # Convert to 64-bit unsigned integer
    unpacked = struct.unpack('>Q', packed)[0]

    # Extract sign (1 bit), exponent (11 bits), and mantissa (52 bits)
    sign = (unpacked >> 63) & 1
    exponent = (unpacked >> 52) & 0x7FF
    mantissa = unpacked & ((1 << 52) - 1)

    return sign, exponent, mantissa

# Function to convert IEEE 754 binary (sign, exponent, mantissa) back to float
def ieee754_to_float(sign, exponent, mantissa):
    # Reassemble the binary representation
    binary_rep = (sign << 63) | (exponent << 52) | mantissa
    # Convert back to float
    packed = struct.pack('>Q', binary_rep)
    return struct.unpack('>d', packed)[0]

# Function to flip bits in the mantissa to achieve exact Hamming distance
def flip_bits_in_mantissa(mantissa, n_flips):
    # Convert mantissa to a binary string
    mantissa_bin = list(f'{mantissa:052b}')

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(mantissa_bin), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        mantissa_bin[idx] = '1' if mantissa_bin[idx] == '0' else '0'

    # Join the list back into a string and convert to integer
    return int(''.join(mantissa_bin), 2)

# Function to modify weights by flipping bits in the mantissa
def modify_weights_mantissa(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight's mantissa
    for i in range(len(flat_weights)):
        # Convert the weight to IEEE 754 components
        sign, exponent, mantissa = float_to_ieee754(flat_weights[i])

        # Flip bits in the mantissa to achieve the specified Hamming distance
        modified_mantissa = flip_bits_in_mantissa(mantissa, hamming_distance)

        # Convert the modified IEEE 754 components back to a float
        flat_weights[i] = ieee754_to_float(sign, exponent, modified_mantissa)

    # Reshape back to the original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 0  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_weights_mantissa(initial_weights[layer_index * 2], hamming_distance=8)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.8748 - loss: 0.4522 - val_accuracy: 0.9620 - val_loss: 0.1255
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.9690 - loss: 0.1014 - val_accuracy: 0.9686 - val_loss: 0.0986
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 41s 6ms/step - accuracy: 0.9794 - loss: 0.0681 - val_accuracy: 0.9731 - val_loss: 0.0832
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.9855 - loss: 0.0451 - val_accuracy: 0.9766 - val_loss: 0.0781
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - accuracy: 0.9897 - loss: 0.0336 - val_accuracy: 0.9768 - val_loss: 0.0775


# **Hamming distance 2 on layer 2(exponent):**




In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# Get the initial weights of the model
initial_weights = model.get_weights()

# Function to decompose a float into sign, exponent, and mantissa
def decompose_float(value):

    # Convert float to 64-bit binary
    binary_repr = format(struct.unpack('!Q', struct.pack('!d', value))[0], '064b')
    sign = binary_repr[0]
    exponent = binary_repr[1:12]  # 11 bits for exponent in IEEE 754
    mantissa = binary_repr[12:]   # 52 bits for mantissa
    return sign, exponent, mantissa

# Function to reconstruct a float from sign, exponent, and mantissa
def reconstruct_float(sign, exponent, mantissa):
    # Reconstruct the binary string
    binary_repr = sign + exponent + mantissa
    # Convert back to float
    return struct.unpack('!d', struct.pack('!Q', int(binary_repr, 2)))[0]

# Function to flip bits in the exponent part to achieve Hamming distance
def flip_bits_exponent(exponent_str, n_flips):
    # Convert the exponent string to a list of characters
    exponent_list = list(exponent_str)

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(exponent_list), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        exponent_list[idx] = '1' if exponent_list[idx] == '0' else '0'

    # Join the list back into a string
    return ''.join(exponent_list)

# Function to modify weights by flipping bits in the exponent only
def modify_exponent_hamming_distance(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight to ensure a Hamming distance on the exponent
    for i in range(len(flat_weights)):
        # Decompose the float into sign, exponent, and mantissa
        sign, exponent, mantissa = decompose_float(flat_weights[i])

        # Flip bits in the exponent to achieve the specified Hamming distance
        modified_exponent = flip_bits_exponent(exponent, hamming_distance)

        # Reconstruct the float using the modified exponent and original sign/mantissa
        flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)

    # Reshape back to original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 1  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_exponent_hamming_distance(initial_weights[layer_index * 2], hamming_distance=2)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


<ipython-input-37-cab7a8298e08>:50: RuntimeWarning: overflow encountered in cast
  flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0998 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 43s 6ms/step - accuracy: 0.0967 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 42s 7ms/step - accuracy: 0.0991 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.0996 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0990 - loss: nan - val_accuracy: 0.0980 - val_loss: nan


# **Hamming distance 2 on layer 2(mantissa):**


In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# # Get the initial weights of the model
initial_weights = model.get_weights()


# Function to convert float to IEEE 754 binary (sign, exponent, mantissa)
def float_to_ieee754(value):
    # Pack the float as a binary string (64-bit)
    packed = struct.pack('>d', value)
    # Convert to 64-bit unsigned integer
    unpacked = struct.unpack('>Q', packed)[0]

    # Extract sign (1 bit), exponent (11 bits), and mantissa (52 bits)
    sign = (unpacked >> 63) & 1
    exponent = (unpacked >> 52) & 0x7FF
    mantissa = unpacked & ((1 << 52) - 1)

    return sign, exponent, mantissa

# Function to convert IEEE 754 binary (sign, exponent, mantissa) back to float
def ieee754_to_float(sign, exponent, mantissa):
    # Reassemble the binary representation
    binary_rep = (sign << 63) | (exponent << 52) | mantissa
    # Convert back to float
    packed = struct.pack('>Q', binary_rep)
    return struct.unpack('>d', packed)[0]

# Function to flip bits in the mantissa to achieve exact Hamming distance
def flip_bits_in_mantissa(mantissa, n_flips):
    # Convert mantissa to a binary string
    mantissa_bin = list(f'{mantissa:052b}')

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(mantissa_bin), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        mantissa_bin[idx] = '1' if mantissa_bin[idx] == '0' else '0'

    # Join the list back into a string and convert to integer
    return int(''.join(mantissa_bin), 2)

# Function to modify weights by flipping bits in the mantissa
def modify_weights_mantissa(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight's mantissa
    for i in range(len(flat_weights)):
        # Convert the weight to IEEE 754 components
        sign, exponent, mantissa = float_to_ieee754(flat_weights[i])

        # Flip bits in the mantissa to achieve the specified Hamming distance
        modified_mantissa = flip_bits_in_mantissa(mantissa, hamming_distance)

        # Convert the modified IEEE 754 components back to a float
        flat_weights[i] = ieee754_to_float(sign, exponent, modified_mantissa)

    # Reshape back to the original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 0  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_weights_mantissa(initial_weights[layer_index * 2], hamming_distance=16)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 23s 6ms/step - accuracy: 0.8736 - loss: 0.4512 - val_accuracy: 0.9614 - val_loss: 0.1207
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 40s 5ms/step - accuracy: 0.9700 - loss: 0.0990 - val_accuracy: 0.9749 - val_loss: 0.0886
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.9797 - loss: 0.0663 - val_accuracy: 0.9743 - val_loss: 0.0768
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.9877 - loss: 0.0419 - val_accuracy: 0.9778 - val_loss: 0.0802
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.9887 - loss: 0.0373 - val_accuracy: 0.9768 - val_loss: 0.0768


# **Hamming distance 4 on layer 2(exponent):**


In [ ]:
import struct
seed = 42
tf.keras.utils.set_random_seed(seed)
tf.config.experimental.enable_op_determinism()

layer_dims = [784 , 256 , 64, 10]
model = Sequential()
input_dim = layer_dims[0]  # Extracting the input layer dimension
# Adding the input layer separately
model.add(Dense(layer_dims[1], activation='sigmoid', input_dim=input_dim))
# Adding the rest of the layers
if len(layer_dims) > 2:
    # Adding the rest of the layers
    for dim in layer_dims[2:]:
        model.add(Dense(dim, activation='sigmoid'))

In [ ]:
# Get the initial weights of the model
initial_weights = model.get_weights()

# Function to decompose a float into sign, exponent, and mantissa
def decompose_float(value):

    # Convert float to 64-bit binary
    binary_repr = format(struct.unpack('!Q', struct.pack('!d', value))[0], '064b')
    sign = binary_repr[0]
    exponent = binary_repr[1:12]  # 11 bits for exponent in IEEE 754
    mantissa = binary_repr[12:]   # 52 bits for mantissa
    return sign, exponent, mantissa

# Function to reconstruct a float from sign, exponent, and mantissa
def reconstruct_float(sign, exponent, mantissa):
    # Reconstruct the binary string
    binary_repr = sign + exponent + mantissa
    # Convert back to float
    return struct.unpack('!d', struct.pack('!Q', int(binary_repr, 2)))[0]

# Function to flip bits in the exponent part to achieve Hamming distance
def flip_bits_exponent(exponent_str, n_flips):
    # Convert the exponent string to a list of characters
    exponent_list = list(exponent_str)

    # Randomly select n_flips positions to flip
    flip_indices = np.random.choice(len(exponent_list), n_flips, replace=False)

    # Flip the bits at the selected indices
    for idx in flip_indices:
        exponent_list[idx] = '1' if exponent_list[idx] == '0' else '0'

    # Join the list back into a string
    return ''.join(exponent_list)

# Function to modify weights by flipping bits in the exponent only
def modify_exponent_hamming_distance(weights, hamming_distance):
    # Flatten the weights to easily manipulate them
    flat_weights = weights.flatten()

    # Modify each weight to ensure a Hamming distance on the exponent
    for i in range(len(flat_weights)):
        # Decompose the float into sign, exponent, and mantissa
        sign, exponent, mantissa = decompose_float(flat_weights[i])

        # Flip bits in the exponent to achieve the specified Hamming distance
        modified_exponent = flip_bits_exponent(exponent, hamming_distance)

        # Reconstruct the float using the modified exponent and original sign/mantissa
        flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)

    # Reshape back to original shape
    return flat_weights.reshape(weights.shape)

# Modify only the weights of the second layer (layer 2)
layer_index = 2  # Index of the second layer (0-based index)
initial_weights[layer_index * 2] = modify_exponent_hamming_distance(initial_weights[layer_index * 2], hamming_distance=8)

# Assign the modified weights back to the model
model.set_weights(initial_weights)


<ipython-input-65-5de1be6331bd>:50: RuntimeWarning: overflow encountered in cast
  flat_weights[i] = reconstruct_float(sign, modified_exponent, mantissa)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.0973 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0995 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.0993 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 39s 5ms/step - accuracy: 0.0977 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.0975 - loss: nan - val_accuracy: 0.0980 - val_loss: nan
